In [1]:
import cv2
import numpy as np
import pandas as pd
import pickle
import xgboost as xgb
from matplotlib import pyplot as plt
import skimage
import os
import sys
import sqlite3
import shutil
from time import time
sys.path.append(os.environ['REPO_DIR'])
from lib.utils import configuration, run
from matplotlib.path import Path
from shapely.geometry import Polygon

In [2]:
def CDF(x):
    x=np.sort(x)
    size=x.shape[0]
    y=np.arange(0,size)/size
    return x,y


def setup_download_from_s3(rel_fp, recursive=True):
    s3_fp = 's3://mousebrainatlas-data/' + rel_fp
    local_fp = os.environ['ROOT_DIR'] + rel_fp

    if os.path.exists(local_fp):
        print('ALREADY DOWNLOADED FILE')
        return

    if recursive:
        run('aws s3 cp --recursive {0} {1}'.format(s3_fp, local_fp))
    else:
        run('aws s3 cp {0} {1}'.format(s3_fp, local_fp))

def setup_upload_from_s3(rel_fp, recursive=True):
    s3_fp = 's3://mousebrainatlas-data/' + rel_fp
    local_fp = os.environ['ROOT_DIR'] + rel_fp

    if recursive:
        run('aws s3 cp --recursive {0} {1}'.format(local_fp, s3_fp))
    else:
        run('aws s3 cp {0} {1}'.format(local_fp, s3_fp))

In [3]:
stack = 'MD594'
fp = os.path.join('CSHL_data_processed', stack, stack + '_sorted_filenames.txt')
setup_download_from_s3(fp, recursive=False)
with open(os.environ['ROOT_DIR']+fp, 'r') as f:
    fn_idx_tuples = [line.strip().split() for line in f.readlines()]
    section_to_filename = {int(idx): fn for fn, idx in fn_idx_tuples}


fname = os.path.join('CSHL_data_processed', stack, 'Annotation.npy')
setup_download_from_s3(fname, recursive=False)
annotation = np.load(os.environ['ROOT_DIR']+fname, allow_pickle = True, encoding='latin1')
contours = pd.DataFrame(annotation)
contours = contours.rename(columns={0:"name", 1:"section", 2:"vertices"})
contours_grouped = contours.groupby('section')
valid_sections = np.sort(contours['section'].unique())

fn = 'CSHL_data_processed/MD589/ThresholdsV2.pkl'
setup_download_from_s3(fn, recursive=False)
thresholds = pickle.load(open(os.environ['ROOT_DIR']+fn,'rb'))

ALREADY DOWNLOADED FILE
ALREADY DOWNLOADED FILE
ALREADY DOWNLOADED FILE


In [4]:
savepath = 'CSHL_cells_mark/'
if not os.path.exists(os.environ['ROOT_DIR']+savepath):
    os.mkdir(os.environ['ROOT_DIR']+savepath)
savepath = savepath+stack+'/'
if not os.path.exists(os.environ['ROOT_DIR']+savepath):
    os.mkdir(os.environ['ROOT_DIR']+savepath)
savepath = savepath+'color/'
if not os.path.exists(os.environ['ROOT_DIR']+savepath):
    os.mkdir(os.environ['ROOT_DIR']+savepath)

resol = 0.46

paired_structures = ['5N', '6N', '7N', '7n', 'Amb', 'LC', 'LRt', 'Pn', 'Tz', 'VLL', 'RMC', \
                     'SNC', 'SNR', '3N', '4N', 'Sp5I', 'Sp5O', 'Sp5C', 'PBG', '10N', 'VCA', 'VCP', 'DC']
singular_structures = ['AP', '12N', 'RtTg', 'SC', 'IC']

all_structures = paired_structures + singular_structures

margin = 500/0.46

In [5]:
len(valid_sections)

265

In [6]:
%%time
raw_images_root = 'CSHL_data_processed/'+stack+'/'+stack+'_prep2_lossless_gray/'
for section in valid_sections:
    img_fn = raw_images_root + section_to_filename[section] + '_prep2_lossless_gray.tif'
#     setup_download_from_s3(img_fn, recursive=False)
    img = cv2.imread(os.environ['ROOT_DIR']+img_fn, 2)
#     whole = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
#     polygons = [(contour['name'], contour['vertices']) \
#             for contour_id, contour in contours_grouped.get_group(section).iterrows()]
#     count = 0    
#     cs = []
#     for contour_id, contour in polygons:
#         structure = contour_id
#         if structure not in all_structures:
#             continue
#         polygon = contour.copy()
#         cs.append(polygon.astype(np.int32))
#         count += 1
#         print(section, structure, count, '/', len(polygons))
#     com = cv2.polylines(whole.copy(), cs, True, [255, 0, 0], 3, lineType=50)
    filename = savepath + str(section) + '.tif'
#     com = cv2.cvtColor(com, cv2.COLOR_BGR2RGB)
    cv2.imwrite(os.environ['ROOT_DIR']+filename, img)    

CPU times: user 14min 26s, sys: 3min 21s, total: 17min 47s
Wall time: 20min 44s


In [8]:
img.dtype

dtype('uint8')

In [5]:
%%time
script_dir = os.environ['REPO_DIR']
! python $script_dir/Cell_mark.py 'MD594' 176 

/Users/kuiqian/Github/venv/shapeology_venv/lib/python3.7/site-packages/sklearn/externals/joblib/externals/cloudpickle/cloudpickle.py:47: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
ALREADY DOWNLOADED FILE
ALREADY DOWNLOADED FILE
ALREADY DOWNLOADED FILE
ALREADY DOWNLOADED FILE
ALREADY DOWNLOADED FILE
ALREADY DOWNLOADED FILE
run cmd= aws s3 cp /Users/kuiqian/BstemAtlasDataBackup/ucsd_brain/CSHL_cells_mark/MD594/Pn/176.png s3://mousebrainatlas-data/CSHL_cells_mark/MD594/Pn/176.png
upload: ../../../BstemAtlasDataBackup/ucsd_brain/CSHL_cells_mark/MD594/Pn/176.png to s3://mousebrainatlas-data/CSHL_cells_mark/MD594/Pn/176.png
176 Pn 1 / 9
run cmd= aws s3 cp /Users/kuiqian/BstemAtlasDataBackup/ucsd_brain/CSHL_cells_mark/MD594/SC/176.png s3://mousebrainatlas-data/CSHL_cells_mark/MD594/SC/176.png
upload: ../../../BstemAtlasDataBackup/ucsd_brain/CSHL_cells_mark/MD594/SC/176.png to s3://mousebrainatlas-da

upload: ../../../BstemAtlasDataBackup/ucsd_brain/CSHL_cells_mark/MD594/DC/176.png to s3://mousebrainatlas-data/CSHL_cells_mark/MD594/DC/176.png
176 DC 25
run cmd= aws s3 cp /Users/kuiqian/BstemAtlasDataBackup/ucsd_brain/CSHL_cells_mark/MD594/AP/176.png s3://mousebrainatlas-data/CSHL_cells_mark/MD594/AP/176.png
upload: ../../../BstemAtlasDataBackup/ucsd_brain/CSHL_cells_mark/MD594/AP/176.png to s3://mousebrainatlas-data/CSHL_cells_mark/MD594/AP/176.png
176 AP 26
run cmd= aws s3 cp /Users/kuiqian/BstemAtlasDataBackup/ucsd_brain/CSHL_cells_mark/MD594/12N/176.png s3://mousebrainatlas-data/CSHL_cells_mark/MD594/12N/176.png
upload: ../../../BstemAtlasDataBackup/ucsd_brain/CSHL_cells_mark/MD594/12N/176.png to s3://mousebrainatlas-data/CSHL_cells_mark/MD594/12N/176.png
176 12N 27
run cmd= aws s3 cp /Users/kuiqian/BstemAtlasDataBackup/ucsd_brain/CSHL_cells_mark/MD594/RtTg/176.png s3://mousebrainatlas-data/CSHL_cells_mark/MD594/RtTg/176.png
upload: ../../../BstemAtlasDataBackup/ucsd_brain/CSHL_c

In [6]:
setup_download_from_s3('CSHL_cells_mark/MD594/')

run cmd= aws s3 cp --recursive s3://mousebrainatlas-data/CSHL_cells_mark/MD594/ /Users/kuiqian/BstemAtlasDataBackup/ucsd_brain/CSHL_cells_mark/MD594/
